In [ ]:
### Installation on Colab (As Of Oct) 
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q -U einops
!pip install -q -U safetensors
!pip install -q -U torch
!pip install -q -U xformers
!pip install -q -U langchain
!pip install -q -U ctransformers[cuda]
!pip install chromadb
!pip install sentence-transformers

https://medium.com/@scholarly360/mistral-7b-complete-guide-on-colab-129fa5e9a04d#id_token=eyJhbGciOiJSUzI1NiIsImtpZCI6ImVkODA2ZjE4NDJiNTg4MDU0YjE4YjY2OWRkMWEwOWE0ZjM2N2FmYzQiLCJ0eXAiOiJKV1QifQ.eyJpc3MiOiJodHRwczovL2FjY291bnRzLmdvb2dsZS5jb20iLCJhenAiOiIyMTYyOTYwMzU4MzQtazFrNnFlMDYwczJ0cDJhMmphbTRsamRjbXMwMHN0dGcuYXBwcy5nb29nbGV1c2VyY29udGVudC5jb20iLCJhdWQiOiIyMTYyOTYwMzU4MzQtazFrNnFlMDYwczJ0cDJhMmphbTRsamRjbXMwMHN0dGcuYXBwcy5nb29nbGV1c2VyY29udGVudC5jb20iLCJzdWIiOiIxMTc2NDg3MDg1NjAzNzAyNjE0MjciLCJlbWFpbCI6InNmbi5jaW90dGlAZ21haWwuY29tIiwiZW1haWxfdmVyaWZpZWQiOnRydWUsIm5iZiI6MTcwODA5NjEwNiwibmFtZSI6IlN0ZWZhbm8gQ2lvdHRpIiwicGljdHVyZSI6Imh0dHBzOi8vbGgzLmdvb2dsZXVzZXJjb250ZW50LmNvbS9hL0FDZzhvY0lNOTNCNDFuaWs1T0ZzYjJ6dFltSmJSTlZjY1FzSVNtV1RmMjlyY2xUbm9RPXM5Ni1jIiwiZ2l2ZW5fbmFtZSI6IlN0ZWZhbm8iLCJmYW1pbHlfbmFtZSI6IkNpb3R0aSIsImxvY2FsZSI6ImVuIiwiaWF0IjoxNzA4MDk2NDA2LCJleHAiOjE3MDgxMDAwMDYsImp0aSI6IjVkMWJmY2Y1NjNkNDMyNDFlMmQ3NDJlNGNlNGRlNDNiY2M2ZjMwZmYifQ.NtT7n-xwRfll37p1RzBGIsqICkuyYBBDH9Av_B6YxwPusKOKEdqDr5tGIts05eYY-cAxCg57E4NvHOZH4Gd3alP1KrRkEYBR4ENHvRy-qMwEiTkd4TjGwX_vlDv64xvyeLCi5hVwEhkH-g8F2wKat02GpnjDjOD98PmPHIq01XjN2E_R50FqoGCSK-kGqfCNDIsKMVsTJB_AoSgfuq0zaYRLi9T9g3LwqVWtywGvtnnWTT4X3_RS_w3O-M4BI8oIXZLoVvbDxXTHrwTOxGu1g6M_6wF2blv5BCvUMJ0mX592mWoGfoNCkYb5qFbmsd2tMvhswz3X_u6_zkm1FDtqXA

Lang-Chain Integration and BitsAndBytes

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import torch
from transformers import BitsAndBytesConfig
from langchain import HuggingFacePipeline
from langchain import PromptTemplate, LLMChain
llm = HuggingFacePipeline(pipeline=pipeline)


c:\Users\stefa\anaconda3\envs\py3aai\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


RuntimeError: Failed to import transformers.pipelines because of the following error (look up to see its traceback):
No module named 'torch'

In [6]:

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)
model_id = "mistralai/Mistral-7B-Instruct-v0.1"


In [5]:

model_4bit = AutoModelForCausalLM.from_pretrained(
    model_id, device_map="auto", quantization_config=quantization_config, )
tokenizer = AutoTokenizer.from_pretrained(model_id)


ImportError: Using `bitsandbytes` 8-bit quantization requires Accelerate: `pip install accelerate` and the latest version of bitsandbytes: `pip install -i https://pypi.org/simple/ bitsandbytes`

In [ ]:

pipeline = pipeline(
    "text-generation",
    model=model_4bit,
    tokenizer=tokenizer,
    use_cache=True,
    device_map="auto",
    max_length=500,
    do_sample=True,
    top_k=5,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.eos_token_id,
)

# Prompt
template = """<s>[INST] You are a helpful, respectful and honest assistant. Answer exactly in few words from the context
Answer the question below from context below :
{context}
{question} [/INST] </s>
"""

question_p = """What is the date for announcement"""
context_p = """ On August 10 said that its arm JSW Neo Energy has agreed to buy a portfolio of 1753 mega watt renewable energy generation capacity from Mytrah Energy India Pvt Ltd for Rs 10,530 crore."""
prompt = PromptTemplate(template=template, input_variables=[
                        "question", "context"])
llm_chain = LLMChain(prompt=prompt, llm=llm)
response = llm_chain.run({"question": question_p, "context": context_p})
response

Retrieval Augmented Generation (RAG) with Chroma-DB

In [9]:
from langchain.schema.document import Document
import chromadb
from chromadb.config import Settings
from langchain.llms import HuggingFacePipeline
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.vectorstores import Chroma


In [10]:

mna_news = """JSW Energy on August 10 said that its arm JSW Neo Energy has agreed to buy a portfolio of 1753 mega watt renewable energy generation capacity from Mytrah Energy India Pvt Ltd for Rs 10,530 crore.
Sajjan Jindal led JSW group had signed an exclusivity agreement with Hyderabad based Mytrah Energy to buy the latter’s wind and solar assets.
This is the largest acquisition by JSW Energy since inception, which comprises 17 special purpose vehicles and 1 ancillary SPV.  The transaction is subject to approval of the Competition Commission of India (CCI) and other customary approvals standard to a transaction of this size, the company said in a release.
With this acquisition, JSW Energy's current  operational generation capacity will go up by over 35 percent from 4,784 MW to 6,537 MW.  Currently about 2,500 MW of under-construction wind and hydro projects which are likely to be commissioned in phases over the next 18-24 months, JSW Energy platform capacity gets enhanced to 9.1 GW – where the share of renewables increases to  65 percent, JSW said in a stock exchange filing. Further, this is likely to help the company in achieving its renewable-led capacity growth target of 10 GW by FY25, well ahead of timelines, the firm added.
"""

In [ ]:


documents = [Document(page_content=mna_news, metadata={"source": "local"})]
#######################
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500, chunk_overlap=20)
all_splits = text_splitter.split_documents(documents)
model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {"device": "cuda"}
embeddings = HuggingFaceEmbeddings(
    model_name=model_name, model_kwargs=model_kwargs)
#######################
vectordb = Chroma.from_documents(
    documents=all_splits, embedding=embeddings, persist_directory="chroma_db")
#######################
retriever = vectordb.as_retriever()
#######################
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    verbose=True
)


In [ ]:


def run_my_rag(qa, query):
    print(f"Query: {query}\n")
    result = qa.run(query)
    print("\nResult: ", result)


# Ask Queries Now
query = """ What company is buyer and seller here """
run_my_rag(qa, query)